## Ceneo Scraper

## Struktura opinii w serwisie ceneo.pl

|składowa|Selektor|zmienna|
|--------|--------|-------|
|identyfikator opinii|["data-entry-id"]|opinion_id|
|autora|.user-post__author-name|author|
|rekomendację|.user-post__author-recomendation >  em|recommend|
|liczbę gwiazdek|.user-post__score-count|stars|
|data wystawienia opinii|span.user-post__published > time::nth-child(1)["datetime"]|opinion_date|
|data zakupu produktu|span.user-post__published > time::nth-child(2)["datetime"]|purchase_date|
|ile osób uznało opinię za przydatną|button.vote-yes["data-total-vote"]|up_vote|
|ile osób uznało opinię za nieprzydatną|button.vote-no["data-total-vote"]|down_vote|
|treść opinii|.user-post__text|content|
|listę wad|div.review-feature__title--negatives~div.review-feature__item|cons|
|listę zalet|div.review-feature__title--positives~div.review-feature__col|pros|

1. Import Bibliotek

In [26]:
selectors={
    "opinion_id": ( None,"data-entry-id"),
    "author": ("span.user-post__author-name",),
    "recommend": (".user-post__author-recomendation > em",),
    "stars": (".user-post__score-count",),
    "content": ("div.user-post__text",),
    "cons": ("div.review-feature__title--negatives ~ div.review-feature__item", None, True),
    "pros":("div.review-feature__title--positives ~ div.review-feature__item", None, True),
    "opinion_date": ("span.user-post__published > time::nth-child(1)","datetime"),
    "purchase_date": ("span.user-post__published > time::nth-child(2)","datetime"),
    "up_vote": ("button.vote-yes","data-total-vote"),
    "down_vote": ("button.vote-no","data-total-vote")
}


In [27]:
import json
import os
import requests
from bs4 import BeautifulSoup

2. Wczytanie kodu produktu  o którym mają zostać wczytane opinie


In [36]:
#product_id="104109736"
product_id=input("Podaj kod produktu: ")
url=f'https://www.ceneo.pl/{product_id}#tab=reviews'

3. Wysłanie do serwera Ceneo.pl żądania dostępu do strony z opiniami o produkcie

In [29]:
response=requests.get(url)
response.status_code

200

4. Przekształcenie kodu html w postaci tekstowej do obiektu reprezentującego strukturę DOM
5. Wydobycie ze struktry DOm znacznikow odpowiadających opiniom konsumentów

In [30]:
page = BeautifulSoup(response.text,"html.parser")
opinions=page.select("div.js_product-review")




3.Pobranie wszystkich opinii o produkcie z serwisu ceneo.pl

In [31]:

all_opinions=[]
while():
    response=requests.get(url)
    page = BeautifulSoup(response.text,"html.parser")
    opinions=page.select("div.js_product-review")
    
    for opinion in opinions:
        single_opinion={
            key: extract(opinion, *value)
                for key,value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url= "https://www.ceneo.pl/"+extract(page, "a.pagination_next","href")
    except TypeError:
        url=None


6. Wydobycie ze struktury znacznika odpowiadającego pojedynczej opinii jej składowych

In [32]:
def extract(ancestor,selector=None, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
        return ancestor.select_one(selector).text.strip()
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()

In [33]:
single_opinion

NameError: name 'single_opinion' is not defined

4. Zapis opinii o produkcie do pliku JSON do katalogu opinions

In [34]:
if not os.path.exists('opinions'):
    os.mkdir('opinions')
with open(f'opinions/{product_id}.json',"w",encoding='UTF-8') as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False )
